# CATBOOST

## Utilização do classificador CatBoost para criar um modelo para previsão de evasão em escolas técnicas.

In [1]:
# Importação dos pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.model_selection import train_test_split
import imblearn
from sklearn.metrics import accuracy_score
from scipy.sparse import csr_matrix
from scipy import sparse
import pickle
from imblearn.over_sampling import SMOTE

In [2]:
# Carrega o arquivo em um dataset.  Apaga a coluna que vem com o dataset.
tecnicoJoinville = pd.read_csv("TECNICOS_Joinville5.csv", encoding = 'latin-1', low_memory = False)
tecnicoJoinville.drop(["Unnamed: 0"], axis = 1, inplace = True)

In [3]:
# Observando as 5 primeiras linhas do dataset
tecnicoJoinville.head()

,produto_educacao,carga_horaria,turno,gratuidade_turma,tipo,online,bairro,cidade,genero,escolaridade,situacao_ocupacional,situacao_matricula,IDADE,empresa
0,AUTOMACAO,1200,Tarde,Mista,PF,Nao,Aventureiro,Joinville,M,Medio_C,Empregado,Concluinte,21,Schulz
1,FABRICACAO,1600,Noite,Mista,PF,Nao,Jardim_Paraiso,Joinville,M,Medio_C,Empregado,Concluinte,18,N_D
2,LOGISTICA,1300,Noite,Mista,PF,Nao,ForaJoinville,SaoFrancisco_Sul,M,Medio_C,Empregado,Concluinte,18,SENAI/SC
3,AUTOMACAO,1600,Tarde,Mista,PF,Nao,Aventureiro,Joinville,M,Medio_C,Desempregado,Concluinte,16,N_D
4,MECANICA,1620,Tarde,Mista,PF,Nao,Ulysses_Guimaraes,Joinville,M,Medio_C,Empregado,Concluinte,17,Schulz


In [4]:
null_value_stats = tecnicoJoinville.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Series([], dtype: int64)

In [5]:
X = tecnicoJoinville.drop('situacao_matricula', axis=1)
y = tecnicoJoinville.situacao_matricula

In [6]:
print(X.dtypes)

categorical_features_indices = np.where(X.dtypes != float)[0]

produto_educacao        object
carga_horaria           object
turno                   object
gratuidade_turma        object
tipo                    object
online                  object
bairro                  object
cidade                  object
genero                  object
escolaridade            object
situacao_ocupacional    object
IDADE                    int64
empresa                 object
dtype: object


In [7]:
categorical_features_indices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [8]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

In [10]:
X_validation

,produto_educacao,carga_horaria,turno,gratuidade_turma,tipo,online,bairro,cidade,genero,escolaridade,situacao_ocupacional,IDADE,empresa
1075,AUTOMACAO,1200,Tarde,Regimental,PF,Nao,Profipo,Joinville,M,Medio_I,Desempregado,17,N_D
1650,MECANICA,1360,Noite,Pago_PF,PF,Sim,Jardim_Paraiso,Joinville,M,Medio_I,Empregado,19,N_D
1861,AUTOMOTIVO,1280,Noite,Pago_PF,PF,Sim,Aventureiro,Joinville,M,Medio_C,Empregado,20,Embraco
941,PLASTICO,1200,Noite,Mista,PF,Nao,Paranaguamirim,Joinville,F,Medio_I,Empregado,18,EBOCAM
331,TEXTIL,1400,Noite,Sem_Gratuidade,PJ_PF,Nao,Aventureiro,Joinville,M,Medio_C,Empregado,32,MALHARIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,QUIMICA,1200,Tarde,Pago_PJ,PF,Nao,America,Joinville,M,Medio_I,Desempregado,16,N_D
561,LOGISTICA,960,Noite,Mista,PF,Sim,Adhemar_Garcia,Joinville,M,Medio_C,Desempregado,21,N_D
909,INFORMATICA,1200,Noite,Sem_Gratuidade,PF,Nao,Anita_Garibaldi,Joinville,M,Medio_I,Desempregado,16,N_D
465,INFORMATICA,1200,Noite,Sem_Gratuidade,PF,Sim,Jardim_Paraiso,Joinville,M,Medio_C,Desempregado,18,N_D


In [9]:
modelCat = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent'
)

In [12]:
modelCat.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_validation, y_validation),
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [14]:
cv_params = modelCat.get_params()
cv_params.update({
    'loss_function': metrics.Logloss()
})
cv_data = cv(
    Pool(X, y, cat_features=categorical_features_indices),
    cv_params,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [15]:
print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-Accuracy-mean']),
    cv_data['test-Accuracy-std'][np.argmax(cv_data['test-Accuracy-mean'])],
    np.argmax(cv_data['test-Accuracy-mean'])
))

Best validation accuracy score: 0.88±0.01 on step 514


In [16]:
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

Precise validation accuracy score: 0.8842028628699206


In [18]:
predictions = modelCat.predict(X_validation)
predictions_probs = modelCat.predict_proba(X_validation)
print(predictions[:10])
print(predictions_probs[:10])

['Desistente' 'Desistente' 'Desistente' 'Desistente' 'Concluinte'
 'Concluinte' 'Desistente' 'Desistente' 'Desistente' 'Concluinte']
[[0.00369347 0.99630653]
 [0.00837246 0.99162754]
 [0.10622523 0.89377477]
 [0.01487341 0.98512659]
 [0.94614345 0.05385655]
 [0.81826717 0.18173283]
 [0.00284734 0.99715266]
 [0.3250672  0.6749328 ]
 [0.01531077 0.98468923]
 [0.85088401 0.14911599]]


In [20]:
# save the model to disk
filename = 'modeloCat.sav'
pickle.dump(modelCat, open(filename, 'wb'))